# Abordagem 1

Numa primeira abordagem usando o N-aryCSP cada atributo do problema será uma variavel, assim como definido na formulação.

In [ ]:
from csp import *
from notebook import psource, pseudocode
import math
import warnings
warnings.filterwarnings("ignore")

## Definição das variaveis a serem usadas na demonstração

**A não consideração da variavel professores para a formulação do horario é uma limitação desta abordagem.**

#### Inputs
S -> (S1, S2, S3, S4) salas
T -> (T1, T2, T3, T4) turmas
D -> (D1, D2, D3, D4, D5) disciplinas
* considerar que disciplinas cada turma tem posteriormente
#### Constantes
* B -> (1, 2, 3, 4) blocos
* DS -> (SEGUNDA, TERCA, QUARTA, ...) dias da semana
Gerar variaveis para cada bloco, dia da semana e turma
Cada turma terá nBlocos * nDiasSemana variáveis para cada 1 dos seguintes pontos:
 
* TD -> Turma Disciplina       #* Dominio = disciplinas em input
* TS -> Turma Sala             #* Dominio = salas em input
* TB -> Turma Bloco            #* Dominio = blocos em constantes
* TDS -> Turma Dia da Semana   #* Dominio = dias da semana em constantes

In [ ]:

constants = {
  "B": set(range(1, 5)),
  # "DS": set(("SEGUNDA", "TERCA", "QUARTA", "QUINTA", "SEXTA"))
  "DS": set(("SEGUNDA", "TERCA", "QUARTA"))
}

inputs = {
  "S": set(("S1", "S2", "S3", "S4", None)),
  "T": set(("T1", "T2")),
  "D": set(("D1", "D2", "D3", None)),
}


### Definição das constraits
Uma das limitações desta abordagem é a existencia  de None no dominio das variaveis. A existencia de None foi necessaria para a resolução de problemas, ams implicou um maior cuidado na criação e tratamento das constraits, de modo a que o enunciado fosse cumprido.

In [ ]:

def max_sum_not_none_constraint(maxValue):
  def _aux(*values):
    sum = 0
    for val in values:
      if val is not None:
        sum += 1
    return sum <= maxValue
  return _aux

def max_ten_classes_per_week_constraint(*values):
  # print("max_ten_classes_per_week_constraint -->", values)
  return max_sum_not_none_constraint(10)(*values)

def max_three_classes_per_day_constraint(*values):
  # print("max_three_classes_per_day_constraint -->", values)
  return max_sum_not_none_constraint(3)(*values)

def both_none_or_none_none_constraint(*values):
  foundNone = False
  foundNotNone = False

  for val in values:
    if val is not None and not foundNotNone:
      foundNotNone = True
    elif foundNotNone and val is None:
      return False
    
    if val is None and not foundNone:
      foundNone = True
    elif foundNone and val is not None:
      return False

  return True

def all_classes_lectured_constraint(classes_to_lecture):
  def _aux(*values):
    # print("all_classes_lectured_constraint.classes_to_lecture", classes_to_lecture)
    # print("all_classes_lectured_constraint.values", values)
    seen = {}
    for val in values:
      if val in classes_to_lecture:
        seen[val] = True

    res = len(seen.keys()) == len(list(filter(lambda x: x is not None, classes_to_lecture)))
    return res
  return _aux

def between_two_to_four_classes_same_room_constraint(*values):
  rooms = {}
  foundRoomWithCountHigherThanTwo = False

  for room in values:
    rooms[room] = rooms.get(room, 0) + 1

    # já encontrou sala com pelo menos 2 aulas atribuidas,
    # todas as outras salas devem ter apenas 1 aula atribuida
    if(foundRoomWithCountHigherThanTwo):
      return False

    if(rooms[room] >= 2):
      foundRoomWithCountHigherThanTwo = True
    elif(rooms[room] >= 4):
      # máximo 4 aulas na mesma sala
      return False

  return True


A função  *tuple_all_diff_constraint* nao foi necessaria mas é relevante a sua existencia para uma futura melhoria do projeto

In [ ]:

def tuple_all_diff_constraint(classes):
  raise Exception("Unused but keep it here because it might be useful")

  print("tuple_all_diff_constraint(classes)", classes)
  def _aux(*values):
    length = len(values)
    splitAt = int(length / len(classes))

    tuples = [values[idx:idx + splitAt] for idx in range(0, len(values), splitAt)]  # split values provided every N items (must be sorted)
                                                                                    # ex: ("TST1", "TBT1", "TDST1", "TST2", "TBT2", "TDST2") becomes:
                                                                                    # (("TST1", "TBT1", "TDST1"), ("TST2", "TBT2", "TDST2"))
    print("tuple_all_diff_constraint.tuples -->", tuples)
    if len(tuples) <= 1:
      return True

    # valores a comparar, usar do primeiro elemento
    first = tuples[0]

    # para cada uma das restantes tuplas, comparar todos os valores
    for i in range(1, len(tuples), 1):
      current = tuples[i]
      for j in range(0, splitAt, 1):
        if current[j] == first[j]:
          return False

    return True 
  return _aux

### Modelação dos dados para o CSP
#### Gerar variáveis e respetivo dominio

In [ ]:
all_classes = list()
domainv2 = {}

for turma in inputs["T"]:
  class_courses = list(map(lambda x: None if x is None else x + turma, inputs["D"]))
  all_classes += class_courses

  for bloco in constants["B"]:
    for weekday in constants["DS"]:
      domainv2[f"TD_DS{weekday}B{bloco}_{turma}"] = class_courses
      # domainv2[f"TD_DS{weekday}B{bloco}_{turma}"] = inputs["D"]
      domainv2[f"TS_DS{weekday}B{bloco}_{turma}"] = inputs["S"]
      domainv2[f"TB_DS{weekday}B{bloco}_{turma}"] = (bloco,)
      domainv2[f"TDS_DS{weekday}B{bloco}_{turma}"] = (weekday, )

#### Gerar constraints

In [ ]:
constraintsv2 = []

for turma in inputs["T"]:
  for bloco in constants["B"]:
    for weekday in constants["DS"]:
      constraintsv2 += [
        # (Disciplina, Sala) ambos none ou nenhum none
        Constraint((f"TD_DS{weekday}B{bloco}_{turma}", f"TS_DS{weekday}B{bloco}_{turma}"), both_none_or_none_none_constraint),
      ]

for bloco in constants["B"]:
  for weekday in constants["DS"]:
    constraint_tupple_diff_vars = ()
    constraint_diff_rooms = ()

    for variable in domainv2:
      if(variable.startswith(f"TS_DS{weekday}B{bloco}")):
        constraint_diff_rooms += (variable,)

    constraintsv2 += [
      # todas as salas para determinado dia da semana / bloco devem ser diferentes.
      Constraint(constraint_diff_rooms, all_diff_constraint),
    ]

for turma in inputs["T"]:
  for weekday in constants["DS"]:
    auxList = ()

    for bloco in constants["B"]:
      auxList += (f"TD_DS{weekday}B{bloco}_{turma}",)
    
    constraintsv2 += [
      # maximo 3 aulas por dia (filtrar por turma/dia_semana)
      Constraint(tuple(filter(lambda x: x in auxList, domainv2.keys())), max_three_classes_per_day_constraint),
    ]

for turma in inputs["T"]:
  constraintsv2 += [
    # maximo 10 aulas por semana (filtrar por turma)
    Constraint(tuple(filter(lambda x: x.startswith("TD_") and x.endswith(turma), domainv2.keys())), max_ten_classes_per_week_constraint),

    # entre 2 a 4 aulas numa sala especifica
    # Constraint(tuple(filter(lambda x: x.startswith("TS_") and x.endswith(turma), domainv2.keys())), between_two_to_four_classes_same_room_constraint),
  ]


# Todas as disciplinas devem ser lecionadas
constraintsv2 += [
  # Constraint(tuple(filter(lambda x: x.startswith("TD_"), domainv2.keys())), all_classes_lectured_constraint(list(filter(lambda x: x is not None, all_classes))))
]


#### Execução
Nesta abordagem usamos o NaryCSP

In [ ]:
class_scheduling = NaryCSP(domainv2, constraintsv2)

# print variables
print("Domain -->", class_scheduling.domains)
print("Variables -->", class_scheduling.variables)
print("Constraints -->", class_scheduling.constraints)

# Result
class_scheduling_solution = ac_search_solver(class_scheduling, arc_heuristic=sat_up)

assert class_scheduling_solution is not None

#### Resultados

In [ ]:

for turma in inputs["T"]:
  print(f"TURMA:\t {turma}\n")

  # for weekday in ("SEGUNDA", "TERCA", "QUARTA", "QUINTA", "SEXTA"):
  for weekday in ("SEGUNDA", "TERCA", "QUARTA"):
    print(f"{weekday}:\n")

    for bloco in constants["B"]:
      print(f"BLOCO {bloco}:\n")

      disciplina = class_scheduling_solution[f"TD_DS{weekday}B{bloco}_{turma}"]
      sala = class_scheduling_solution[f"TS_DS{weekday}B{bloco}_{turma}"]

      print(f"SALA:\t{sala}")
      print(f"DISCIPLINA:\t{disciplina}")
      print("\n")


# Abordagem 2



## Definição das variaveis a serem usadas na demonstração

**Nesta abodargem as salas não são consideradas(são uma parte da disciplina) e os professores estao associados há diciplina.**

#### Classes 

In [ ]:

class Professor:
  def __init__(self, name, availableTimeSlots):
    self.name = name
    self.availableTimeSlots = availableTimeSlots

  def __str__(self):
    return f"PROF({self.name})"

  def __repr__(self):
    return self.__str__()

class Room:
  def __init__(self, name):
    self.name = name

  def __str__(self):
    return f"ROOM({self.name})"

  def __repr__(self):
    return self.__str__()

class TimeSlot:
  def __init__(self, id):
    self.id = id

  def __str__(self):
    return f"T({self.id})"

  def __repr__(self):
    return self.__str__()

class CurricularUnit:
  def __init__(self, name, professor, className):
    self.name = name
    self.professor = professor
    self.className = className
    self.timeslot = None
    self.room = None

  def __str__(self):
    timeslot_id = self.timeslot.id if self.timeslot is not None else None
    room_name = self.room.name if self.room is not None else None
    return f"UC({self.name}, {timeslot_id}, {room_name})"

  def __repr__(self):
    return self.__str__()



#### Inputs & Mapeamento dos vizinhos & Função de Constrait

In [ ]:
def ClassPlan():
  # inputs:
  courses = [
    {
      "name": "T1",
      "disciplinas": [
        {
          "name": "D1",
          "professor": "P1",
          "availableWeekdays": ("SEGUNDA", "TERCA")
        },
        {
          "name": "D2",
          "professor": "P2",
          "availableWeekdays": ("SEGUNDA", "SEXTA")
        },
        {
          "name": "D3",
          "professor": "P3",
          "availableWeekdays": ("QUARTA", "QUINTA")
        },
        {
          "name": "D4",
          "professor": "P4",
          "availableWeekdays": ("TERÇA", "SEXTA")
        },
        {
          "name": "D5",
          "professor": "P5",
          "availableWeekdays": ("QUARTA", "QUINTA")
        },
      ]
    },
    {
      "name": "T2",
      "disciplinas": [
        {
          "name": "U1",
          "professor": "P1",
          "availableWeekdays": ("QUARTA", "QUINTA")
        },
        {
          "name": "U2",
          "professor": "P2",
          "availableWeekdays": ("TERÇA", "SEXTA")
        },
        {
          "name": "U3",
          "professor": "P3",
          "availableWeekdays": ("QUARTA", "QUINTA")
        },
        {
          "name": "U4",
          "professor": "P4",
          "availableWeekdays": ("SEGUNDA", "SEXTA")
        },
        {
          "name": "U5",
          "professor": "P5",
          "availableWeekdays": ("SEGUNDA", "TERCA")
        },
      ]
    },                                               
  ]

  _rooms = set(("S1", "S2", "S3", "S4", "S5"))

  # constants:
  weekdays = set(("SEGUNDA", "TERCA", "QUARTA", "QUINTA", "SEXTA"))
  _timeslots = set(range(1, 4 * len(weekdays) + 1))

  # Initialize variables
  # 1. Rooms
  rooms = set()
  for room in _rooms:
    rooms.add(Room(room))

  # 2. Professors & CurricularUnits
  professors = set()
  curricular_units = set()
  for course in courses:
    for unit in course["disciplinas"]:
      availableTimeSlots = set()

      for weekday in unit["availableWeekdays"]:
        if weekday == "SEGUNDA":
          availableTimeSlots.add(1)
          availableTimeSlots.add(2)
          availableTimeSlots.add(3)
          availableTimeSlots.add(4)
        elif weekday == "TERCA":
          availableTimeSlots.add(5)
          availableTimeSlots.add(6)
          availableTimeSlots.add(7)
          availableTimeSlots.add(8)
        elif weekday == "QUARTA":
          availableTimeSlots.add(9)
          availableTimeSlots.add(10)
          availableTimeSlots.add(11)
          availableTimeSlots.add(12)
        elif weekday == "QUINTA":
          availableTimeSlots.add(13)
          availableTimeSlots.add(14)
          availableTimeSlots.add(15)
          availableTimeSlots.add(16)
        elif weekday == "SEXTA":
          availableTimeSlots.add(17)
          availableTimeSlots.add(18)
          availableTimeSlots.add(19)
          availableTimeSlots.add(20)

      p = Professor(unit["professor"], availableTimeSlots)
      professors.add(p)

      u1 = CurricularUnit(unit["name"] + "1", p, course["name"])
      curricular_units.add(u1)
      u2 = CurricularUnit(unit["name"] + "2", p, course["name"])
      curricular_units.add(u2)

  # 3. Timeslots
  timeslots = set()
  for ts in _timeslots:
    t = TimeSlot(ts)
    timeslots.add(t)

  domain = {}

  for uc in curricular_units:
    domain[uc.name] = (uc,)
    domain[uc.name + "_TS"] = timeslots
    domain[uc.name + "_ROOM"] = rooms

  neighborstr = ""

  # neighbors curricular_units:
  # a. timeslots per course
  for course in courses:
    clone = list(filter(lambda x: x.className == course["name"], curricular_units))
    while len(clone):
      uc = clone.pop(0)
      neighborstr += f"{uc.name + '_TS'}: {' '.join(map(lambda x: x.name + '_TS', clone))};"

  # b. unnocupied room if same timeslot & curricular unit validate
  clone = list(curricular_units)
  while len(clone):
    uc = clone.pop(0)
    neighborstr += f"{uc.name}: {' '.join(map(lambda x: x.name, clone))};"
    # neighborstr += f"{uc.name}_ROOM: {' '.join(map(lambda x: x.name + '_ROOM', clone))};"

  neighborstr = neighborstr[:-1] # remover last ';'
  neighbors = parse_neighbors(neighborstr)

  # Utilitary functions
  def get_curricular_unit(name) -> CurricularUnit:
    return domain[name][0]

  def all_curricular_units_with_timeslot() -> bool:
    for uc in curricular_units:
      if uc.timeslot is None:
        return False
    return True

  counter = {}

  def classplan_constraint(A, a, B, b, recurse=0, _neighbors = neighbors):
    print(f"classplan_constraint({A}, {a}, {B}, {b}, {recurse})")
    
    if(type(a) == TimeSlot and type(b) == TimeSlot):
      aUnit = get_curricular_unit(A.split('_')[0])
      bUnit = get_curricular_unit(B.split('_')[0])

      if(a != b):
        # Check if uc professor has timeslot in availableTimeSlots
        if a.id not in aUnit.professor.availableTimeSlots:
          return False

        if b.id not in bUnit.professor.availableTimeSlots:
          return False

        # Check for timeslot A day how many are assigned (max 3)
        # 1 2 3 4 |5 6 7 8| 9 10 ... 20 TS por turma
        # 3/4 = 0.75 = 1 (SEGUNDA)
        # 6/4 = 1.5 = 2  (TERCA)
        # 10/4 = 2.5 = 3 (QUARTA) 
        desiredWeekDayA = math.ceil(a.id / 4)
        desiredWeekDayB = math.ceil(b.id / 4)

        if desiredWeekDayA == desiredWeekDayB:
          counter[desiredWeekDayA] = counter.get(desiredWeekDayA, 0)
          if counter[desiredWeekDayA] + 2 > 3:
            return False
        else:
          counter[desiredWeekDayA] = counter.get(desiredWeekDayA, 0)
          counter[desiredWeekDayB] = counter.get(desiredWeekDayB, 0)
          if counter[desiredWeekDayA] + 1 > 3:
            return False
          if counter[desiredWeekDayB] + 1 > 3:
            return False

        counter[desiredWeekDayA] = counter[desiredWeekDayA] + 1
        counter[desiredWeekDayB] = counter[desiredWeekDayB] + 1

        # Assign timeslot to curricular units
        aUnit.timeslot = a
        bUnit.timeslot = b

        # Read all ucs assigned to specified timeslot (if all assigned)
        if all_curricular_units_with_timeslot():
          # Clear previous
          for uc in curricular_units:
            key = uc.name + "_ROOM"
            if key in neighbors:
              neighbors.pop(key)

          # Foreach timeslot assigned, build neighbors
          aux_str = ""
          for ts in timeslots:
            units_slots = list(filter(lambda x: x.timeslot == ts, curricular_units))

            if len(units_slots) == 0:
              continue

            while len(units_slots):
              uc = units_slots.pop(0)
              aux_str += f"{uc.name}_ROOM: {' '.join(map(lambda x: x.name + '_ROOM', units_slots))};"

          aux_str = aux_str[:-1] # remover last ';'
          
          if len(aux_str.strip()) > 0:
            new_neighbors = parse_neighbors(aux_str)
            # print("NEW_NEIGHBORS", new_neighbors)
            for n in new_neighbors:
              _neighbors[n] = _neighbors.get(n, list())
              for item in new_neighbors[n]:
                if item not in _neighbors[n]:
                  _neighbors[n].append(item)
            # print("NEIGHBORS", _neighbors)
          return True

      return False

    if type(a) == Room and type(b) == Room:
      return True
      aUnit = get_curricular_unit(A.split('_')[0])
      bUnit = get_curricular_unit(B.split('_')[0])

      if a != b:
        aUnit.room = a
        bUnit.room = b
        return True
      return False

    if type(a) == CurricularUnit and type(b) == CurricularUnit:
      if(
        a.timeslot is not None and
        b.timeslot is not None and
        a.timeslot == b.timeslot and
        a.professor == b.professor
      ):
        # professor occupied
        print("PROFESSOR OCCUPIED")
        return False

      if(
        a.timeslot is not None and
        b.timeslot is not None and
        a.room is not None and
        b.room is not None and
        a.timeslot == b.timeslot and
        a.room == b.room
      ):
        # room occupied
        print("ROOM OCCUPIED")
        return False

      return True

    if(recurse == 0):
      return classplan_constraint(B, b, A, a, 1, _neighbors)

  print(neighbors)
  variables = domain.keys()
  return CSP(variables, domain, neighbors, classplan_constraint)

#### Resolução 

In [ ]:

def solve_classplan():
  cp = ClassPlan()
  res = backtracking_search(
    csp=cp,
    select_unassigned_variable=mrv, 
    # select_unassigned_variable=first_unassigned_variable, 
    order_domain_values=lcv, 
    # order_domain_values=unordered_domain_values, 
    inference=forward_checking
  )
  print(res)

solve_classplan()